# Credit Risk Resampling Techniques

In [42]:
import warnings
warnings.filterwarnings('ignore')

In [43]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from imblearn.metrics import classification_report_imbalanced


# Read the CSV and Perform Basic Data Cleaning

In [44]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [45]:
# Load the data
file_path = Path('./data/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [46]:
label_encoder = LabelEncoder()
label_encoder.fit(df["verification_status"])
df["verification_status_le"] = label_encoder.transform(df["verification_status"])
label_encoder.fit(df["home_ownership"])
df["home_ownership_le"] = label_encoder.transform(df["home_ownership"])
label_encoder.fit(df["issue_d"])
df["issue_d_le"] = label_encoder.transform(df["issue_d"])
df.drop(['verification_status', 'home_ownership', 'issue_d'], axis=1, inplace=True)
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,...,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,verification_status_le,home_ownership_le,issue_d_le
0,10500.0,0.1719,375.35,66000.0,low_risk,n,27.24,0.0,0.0,8.0,...,0.0,65687.0,38199.0,2000.0,61987.0,N,N,1,3,2
1,25000.0,0.2000,929.09,105000.0,low_risk,n,20.23,0.0,0.0,17.0,...,0.0,271427.0,60641.0,41200.0,49197.0,N,N,2,1,2
2,20000.0,0.2000,529.88,56000.0,low_risk,n,24.26,0.0,0.0,8.0,...,0.0,60644.0,45684.0,7500.0,43144.0,N,N,2,1,2
3,10000.0,0.1640,353.55,92000.0,low_risk,n,31.44,0.0,1.0,10.0,...,0.0,99506.0,68784.0,19700.0,76506.0,N,N,2,3,2
4,22000.0,0.1474,520.39,52000.0,low_risk,n,18.76,0.0,1.0,14.0,...,0.0,219750.0,25919.0,27600.0,20000.0,N,N,0,1,2


In [47]:
df_encoded = pd.get_dummies(
                                        df, columns=['application_type',
                                         'initial_list_status',
                                         'next_pymnt_d','pymnt_plan',
                                         'hardship_flag',
                                         'debt_settlement_flag']
                           )
df_encoded.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,issue_d_le,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,2,1,0,0,1,0,1,1,1,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,2,1,0,0,1,0,1,1,1,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,2,1,0,0,1,0,1,1,1,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,2,1,0,0,1,0,1,1,1,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,2,1,0,0,1,0,1,1,1,1


In [48]:
data=Path('./data/encoded.csv')
df_encoded.to_csv(data)

# Split the Data into Training and Testing

In [49]:
# Create our features
X = df_encoded.copy()
X.drop("loan_status", axis=1, inplace=True)

# Create our target
y = df_encoded['loan_status']
y.rename('loan_status')
y.shape

(68817,)

In [50]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_le,application_type_Individual,application_type_Joint App,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,pymnt_plan_n,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.805542,0.860340,0.139660,0.123879,0.876121,0.383161,0.616839,1.0,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.714932,0.346637,0.346637,0.329446,0.329446,0.486161,0.486161,0.0,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.0,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.0,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0


In [51]:
data_scaler = StandardScaler()
data_scaler.fit(X)
X = data_scaler.transform(X)
X[:1]

array([[-0.60109272,  0.91798436, -0.36555835, -0.1921948 ,  0.27040056,
        -0.30314253, -0.65649151, -0.76165917, -0.37420432, -0.73252183,
        -0.8658573 , -0.58124513, -0.58081958, -0.50046544, -0.50029335,
        -0.3228639 , -0.89960419, -0.03227363,  0.        ,  0.        ,
        -0.17383528, -0.12011247,  0.        ,  0.        , -0.00589695,
        -0.72692228,  1.67997825,  0.2392533 ,  1.19137226,  1.35299632,
        -0.80895603, -0.13418715, -0.45717571, -0.16310175, -0.24864022,
        -0.83203994,  0.12422197, -1.04213361, -0.24758239, -0.59166015,
         0.75578041,  0.37274923, -0.59052419, -0.80915288,  1.04494582,
        -0.07820908, -0.00778953, -0.56994022, -0.7981628 , -0.64845959,
        -0.66313947, -0.8249868 , -0.14674317, -1.10824038, -0.33490355,
        -1.13862411, -1.05103022, -1.27998521, -1.40011594,  0.06599699,
        -1.11347944, -1.31043257, -1.0519437 , -0.75930693,  0.        ,
         0.        , -0.13347217,  0.41138923, -1.1

In [52]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [53]:
# Create X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [54]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
sample= balanced_accuracy_score(y_test, y_pred)
print(sample)

0.5197727472700497


In [55]:
cm=confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Pred High Risk", "Pred Low Risk"]
)
print("Confusion Matrix")
display(cm_df)
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))


Confusion Matrix


,Pred High Risk,Pred Low Risk
Actual High Risk,4,97
Actual Low Risk,1,17103


Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.80      0.04      1.00      0.08      0.20      0.04       101
   low_risk       0.99      1.00      0.04      1.00      0.20      0.04     17104

avg / total       0.99      0.99      0.05      0.99      0.20      0.04     17205



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [56]:
# Resample the training data with the RandomOversampler


ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [57]:
# Train the Logistic Regression model using the resampled data


model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [58]:
# Calculated the balanced accuracy score
ROS=balanced_accuracy_score(y_test, y_pred)
print(ROS)

0.7273520061313896


In [59]:
# Display the confusion matrix
cm=confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Pred High Risk", "Pred Low Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Pred High Risk,Pred Low Risk
Actual High Risk,70,31
Actual Low Risk,4077,13027


In [60]:
# Print the imbalanced classification report

print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))


Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.76      0.03      0.73      0.52       101
   low_risk       1.00      0.76      0.69      0.86      0.73      0.53     17104

avg / total       0.99      0.76      0.69      0.86      0.73      0.53     17205



### SMOTE Oversampling

In [61]:
# Resample the training data with SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train, y_train
)

In [62]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [63]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
smote=balanced_accuracy_score(y_test, y_pred)
print(smote)

0.7232784989209866


In [64]:
# Display the confusion matrix
cm=confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Pred High Risk", "Pred Low Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Pred High Risk,Pred Low Risk
Actual High Risk,69,32
Actual Low Risk,4047,13057


In [65]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))


Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.68      0.76      0.03      0.72      0.52       101
   low_risk       1.00      0.76      0.68      0.86      0.72      0.53     17104

avg / total       0.99      0.76      0.68      0.86      0.72      0.53     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [66]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [67]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [68]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
RUS=balanced_accuracy_score(y_test, y_pred)
print(RUS)

0.7206510086228455


In [69]:
# Display the confusion matrix
cm=confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Pred High Risk", "Pred Low Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Pred High Risk,Pred Low Risk
Actual High Risk,78,23
Actual Low Risk,5661,11443


In [70]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.77      0.67      0.03      0.72      0.52       101
   low_risk       1.00      0.67      0.77      0.80      0.72      0.51     17104

avg / total       0.99      0.67      0.77      0.80      0.72      0.51     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [71]:
# Resample the training data with SMOTEENN
sm = SMOTEENN(random_state=1)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 51366, 'low_risk': 47084})

In [72]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [73]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
combo=balanced_accuracy_score(y_test, y_pred)
print(combo)

0.7169743167020163


In [74]:
# Display the confusion matrix
cm=confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual High Risk", "Actual Low Risk"], columns=["Pred High Risk", "Pred Low Risk"]
)
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Pred High Risk,Pred Low Risk
Actual High Risk,70,31
Actual Low Risk,4432,12672


In [75]:
# Print the imbalanced classification report
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.69      0.74      0.03      0.72      0.51       101
   low_risk       1.00      0.74      0.69      0.85      0.72      0.52     17104

avg / total       0.99      0.74      0.69      0.85      0.72      0.52     17205



In [76]:
print(f"Accuracy Score RF sample: {sample}")
print(f"Accuracy Score RF naive over: {ROS}")
print(f"Accuracy Score RF SMOTE: {smote}")
print(f"Accuracy Score RF under: {RUS}")
print(f"Accuracy Score RF combo: {combo}")


Accuracy Score RF sample: 0.5197727472700497
Accuracy Score RF naive over: 0.7273520061313896
Accuracy Score RF SMOTE: 0.7232784989209866
Accuracy Score RF under: 0.7206510086228455
Accuracy Score RF combo: 0.7169743167020163
